In [2]:
# import module
import pandas as pd

In [3]:
# set up csv read path
df = pd.read_csv("purchase_data.csv")
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# total players
totalplayers = len(df["SN"].unique())
df_totalplayers = pd.DataFrame({"Total Players":[totalplayers]})
df_totalplayers

,Total Players
0,576


In [5]:
# obtain values for: number of unique items; average price; number of purchases; total revenue

unique_items = len(df["Item Name"].unique())
# unique_items

average_price = round(df["Price"].mean(),2)
# average_price

purchase_number = len(df)
# purchase_number

total_revenue = df["Price"].sum()
# total_revenue

summary_basic = pd.DataFrame({
    "Number of Unique Items": [unique_items],
    "Average Price": [average_price],
    "Number of Purchases": [purchase_number],
    "Total Revenue": [total_revenue]
})

# map $ to money values
summary_basic["Average Price"] = summary_basic["Average Price"].map("${:.2f}".format)
summary_basic["Total Revenue"] = summary_basic["Total Revenue"].map("${:.2f}".format)

summary_basic

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [6]:
# gender demographics

unique_players = df.drop_duplicates(["SN"])
# gender["Gender"].value_counts()

# individual gender values
count_male = len(unique_players.loc[unique_players["Gender"] == "Male"])
count_female = len(unique_players.loc[unique_players["Gender"] == "Female"])
count_other = len(unique_players.loc[unique_players["Gender"] == "Other / Non-Disclosed"])

# summary table, calcluating % using totalplayers from above
summary_gender = pd.DataFrame({
    "Gender": ["Male", "Female", "Other / Non-Disclosed"],
    "Total Count": [count_male, count_female, count_other],
    "Percentage of Players": [(count_male / totalplayers), (count_female / totalplayers), (count_other / totalplayers)]
})

# re-mapping that column as 2dp %
summary_gender["Percentage of Players"] = summary_gender["Percentage of Players"].map("{:.2%}".format)

# setting gender as index
summary_gender = summary_gender.set_index("Gender")
summary_gender

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
# purchase stats by gender

# value variables. example of expanded: df.loc[df["Gender"] == "Male"]["Price"].sum()
male = df.loc[df["Gender"] == "Male"]
female = df.loc[df["Gender"] == "Female"]
other = df.loc[df["Gender"] == "Other / Non-Disclosed"]

summary_genbuy = pd.DataFrame({
    "Gender": ["Female", "Male", "Other / Non-Disclosed"],
    "Purchase Count": [len(female), len(male), len(other)],
    "Average Purchase Price": [female["Price"].mean(), male["Price"].mean(), other["Price"].mean()],
    "Total Purchase Value": [female["Price"].sum(), male["Price"].sum(), other["Price"].sum()],
    "Avg Total Purchase per Person": [female["Price"].sum() / count_female, male["Price"].sum() / count_male, other["Price"].sum() / count_other]
})

# map $ to money values
summary_genbuy["Average Purchase Price"] = summary_genbuy["Average Purchase Price"].map("${:.2f}".format)
summary_genbuy["Total Purchase Value"] = summary_genbuy["Total Purchase Value"].map("${:,.2f}".format)
summary_genbuy["Avg Total Purchase per Person"] = summary_genbuy["Avg Total Purchase per Person"].map("${:.2f}".format)

# set gender as index
summary_genbuy = summary_genbuy.set_index("Gender")

summary_genbuy

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
# age bins

agebins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 100]
agebinlabels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

unique_players["Age Group"] = pd.cut(unique_players["Age"], agebins, labels=agebinlabels, include_lowest=True)
summary_age_table = unique_players.groupby("Age Group").count()
summary_age_table

# make df of it
summary_age = summary_age_table[["Purchase ID"]]
summary_age = summary_age.rename(columns={"Purchase ID":"Total Count"})

# add % column, format it
summary_age["Percentage of Players"] = summary_age["Total Count"] / summary_age["Total Count"].sum()
summary_age["Percentage of Players"] = summary_age["Percentage of Players"].map("{:.2%}".format)
summary_age


<ipython-input-8-a17c3efb2aa5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_players["Age Group"] = pd.cut(unique_players["Age"], agebins, labels=agebinlabels, include_lowest=True)


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [9]:
# purchase stats by age
# bin ages onto main df instead of uniques

df["Age Ranges"] = pd.cut(df["Age"], agebins, labels=agebinlabels, include_lowest=True)
# df.head()

In [10]:
summary_agebuy = pd.DataFrame({
    "Age Ranges": [x for x in agebinlabels],
    "Purchase Count": [len(df.loc[df["Age Ranges"] == x]) for x in agebinlabels],
    "Average Purchase Price": [df.loc[df["Age Ranges"] == x]["Price"].mean() for x in agebinlabels],
    "Total Purchase Value": [df.loc[df["Age Ranges"] == x]["Price"].sum() for x in agebinlabels],
    "Avg Total Purchase per Person": [df.loc[df["Age Ranges"] == x]["Price"].sum() / len(unique_players.loc[unique_players["Age Group"] == x]) for x in agebinlabels]
})

# format $
summary_agebuy["Average Purchase Price"] = summary_agebuy["Average Purchase Price"].map("${:.2f}".format)
summary_agebuy["Total Purchase Value"] = summary_agebuy["Total Purchase Value"].map("${:,.2f}".format)
summary_agebuy["Avg Total Purchase per Person"] = summary_agebuy["Avg Total Purchase per Person"].map("${:.2f}".format)

# set index to age
summary_agebuy = summary_agebuy.set_index(["Age Ranges"])

summary_agebuy

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [11]:
# big spenders

playerlist = unique_players["SN"]

summary_bigspenders = pd.DataFrame({
    "SN": [x for x in playerlist],
    "Purchase Count": [len(df.loc[df["SN"] == x]) for x in playerlist],
    "Average Purchase Price": [df.loc[df["SN"] == x]["Price"].mean() for x in playerlist],
    "Total Purchase Value": [df.loc[df["SN"] == x]["Price"].sum() for x in playerlist],
})

# set index
summary_bigspenders = summary_bigspenders.set_index(["SN"])

# sort by total purchase value, need to redefine to keep this way
summary_bigspenders = summary_bigspenders.sort_values(["Total Purchase Value"], ascending=False)

# now we can reformat $
summary_bigspenders["Average Purchase Price"] = summary_bigspenders["Average Purchase Price"].map("${:.2f}".format)
summary_bigspenders["Total Purchase Value"] = summary_bigspenders["Total Purchase Value"].map("${:.2f}".format)

# get top 5 values
summary_bigspenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [16]:
# popular items

itemidlist = df.drop_duplicates(["Item Name"])["Item ID"]
itemlist = df.drop_duplicates(["Item Name"])["Item Name"]

summary_items = pd.DataFrame({
    "Item ID": [x for x in itemidlist],
    "Item Name": [x for x in itemlist],
    "Purchase Count": [len(df.loc[df["Item Name"] == x]) for x in itemlist],
    "Item Price": [df.loc[df["Item Name"] == x]["Price"].mean() for x in itemlist],
    "Total Purchase Value": [df.loc[df["Item Name"] == x]["Price"].sum() for x in itemlist]
})

# set index
summary_items = summary_items.set_index(["Item ID", "Item Name"])

# sort by purchase count (descending). set as new variable so the same (unmodified) table above can be used in the next section
summary_items_purchase = summary_items.sort_values(["Purchase Count"], ascending=False)

# format
summary_items_purchase["Item Price"] = summary_items_purchase["Item Price"].map("${:.2f}".format)
summary_items_purchase["Total Purchase Value"] = summary_items_purchase["Total Purchase Value"].map("${:.2f}".format)

# top 5 values
summary_items_purchase.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22


In [17]:
# profitable items

# yoink table from above and sort by total purchase value
summary_items_value = summary_items.sort_values(["Total Purchase Value"], ascending=False)

# format
summary_items_value["Item Price"] = summary_items_value["Item Price"].map("${:.2f}".format)
summary_items_value["Total Purchase Value"] = summary_items_value["Total Purchase Value"].map("${:.2f}".format)

# top 5 values
summary_items_value.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


# Summary Report

One thing to note is that this is a dataset of total purchases, and not total players; there are no gender/age demographics for players who are not making any in-game transactions at all, which may deliver different results. If we’re getting into specifics, there is also no data for the player level/time played for each account, nor the stats/levels of the items themselves.

Female and non-binary/unspecified players, while making up a smaller portion of the playerbase, tend to spend more on average than their male counterparts (both by average purchase price and average total purchase per person). Females also make more purchases relative to their playerbase proportion – making 14.49% of all total purchases while representing 14.06% of the playerbase.

20-24 years of age is the largest demographic of players, and the numbers fall off sharply after that. It seems to be the sweet spot between having the money to afford access to a computer good enough to run the game, and having enough free time to actually play it (before the onset of responsibilities like full time work, building a family etc). The content of the game is likely also targeted at this demographic for the above reasons.
When looking at the most popular items to buy, the top of the table is filled with a combination of cheap ($1-2) and expensive ($4-5) items. For items that were purchased 7 or more times, 14/23 of these fell into the above categories. The price is likely associated with the strength of the item, so the majority of the purchasing is done by new players looking to upgrade their lowly starter items, and veteran players approaching end-game content and desiring the best items available.